In [1]:
### allows to direct changes in the packages to be immidiately visibe in the notebook 
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
sys.path.append(os.getcwd() + '/../..')
sys.path.append(os.getcwd() + '/..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from datetime import datetime
import helper_v1 as helper

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import add_changepoints_to_plot
from matplotlib import pyplot as plt 


In [55]:
df_cell = pd.read_csv('./data/cell/traffic_cells.csv', sep='|', encoding="UTF-8",
                    warn_bad_lines=False, error_bad_lines=False, header=0, index_col=0)

C:\ProgramData\Anaconda3\envs\shared\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [56]:
df_cell.head()

,cell_key,quarter_hour,traffic_active_user_dl_avg,chmeas_prb_dl_avail,chmeas_prb_dl_used_avg,thrp_bits_dl,thrp_time_dl,dt
0,4,00:15,0.400,100.0,14.291,3.324068e+09,223320.0,20190520
1,4,02:30,0.229,100.0,3.360,5.969072e+08,103160.0,20190520
2,4,07:15,0.067,100.0,3.468,1.853983e+09,34478.0,20190520
3,4,11:15,0.115,100.0,4.786,7.638756e+08,38768.0,20190520
4,5,09:30,0.199,100.0,5.074,2.051302e+09,84535.0,20190520


## Lets prepare the data and add additional fields/information/columns

#### Lets combine the date & time columns together

In [57]:
df_cell['dt'] = df_cell['dt'].astype(str)
df_cell['quarter_hour'] = df_cell['quarter_hour']
df_cell['DateTime'] = df_cell['dt'] +' ' + df_cell['quarter_hour']
df_cell['DateTime'] = pd.to_datetime(df_cell['DateTime'], format='%Y%m%d %H:%M:%S.%f')
df_cell = df_cell.sort_values(['cell_key','DateTime'], ascending=True)

In [58]:
df_cell

,cell_key,quarter_hour,traffic_active_user_dl_avg,chmeas_prb_dl_avail,chmeas_prb_dl_used_avg,thrp_bits_dl,thrp_time_dl,dt,DateTime
1057716,0,00:00,0.451,100.0,15.076,6.825372e+09,181509.0,20190102,2019-01-02 00:00:00
1049620,0,00:15,0.427,100.0,17.461,7.944895e+09,234084.0,20190102,2019-01-02 00:15:00
725344,0,00:30,0.264,100.0,9.163,2.130582e+09,103033.0,20190102,2019-01-02 00:30:00
1072162,0,00:45,0.274,100.0,12.535,2.550227e+09,139956.0,20190102,2019-01-02 00:45:00
1078715,0,01:00,0.360,100.0,13.531,3.145219e+09,150096.0,20190102,2019-01-02 01:00:00
...,...,...,...,...,...,...,...,...,...
170217,80,22:45,0.207,100.0,7.253,2.169892e+09,87229.0,20190601,2019-06-01 22:45:00
174118,80,23:00,0.218,100.0,7.670,2.771119e+09,93755.0,20190601,2019-06-01 23:00:00
172113,80,23:15,0.114,100.0,4.547,1.492942e+09,58496.0,20190601,2019-06-01 23:15:00
174119,80,23:30,0.089,100.0,3.470,9.231914e+08,36180.0,20190601,2019-06-01 23:30:00


#### Out task is to predict the "chmeas_prb_dl_used_avg" for the next three days

In [124]:
df_data = df_cell[['chmeas_prb_dl_used_avg', 'DateTime']]
df_data.columns = ['y', 'ds']
df_data.index = df_data['ds']

#### Day of a Week gives importatn insight here since traffic on weekends are higher compared to normal week days therefore, we explicitly specify day of week 

In [125]:
df_data['week_days'] = df_data['ds'].apply(helper.is_weekend)
df_data['week_ends'] = ~df_data['ds'].apply(helper.is_weekend)

<ipython-input-125-30a421570536>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['week_days'] = df_data['ds'].apply(helper.is_weekend)
<ipython-input-125-30a421570536>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['week_ends'] = ~df_data['ds'].apply(helper.is_weekend)


In [126]:
df_data

,y,ds,week_days,week_ends
ds,,,,
2019-01-02 00:00:00,15.076,2019-01-02 00:00:00,True,False
2019-01-02 00:15:00,17.461,2019-01-02 00:15:00,True,False
2019-01-02 00:30:00,9.163,2019-01-02 00:30:00,True,False
2019-01-02 00:45:00,12.535,2019-01-02 00:45:00,True,False
2019-01-02 01:00:00,13.531,2019-01-02 01:00:00,True,False
...,...,...,...,...
2019-06-01 22:45:00,7.253,2019-06-01 22:45:00,False,True
2019-06-01 23:00:00,7.670,2019-06-01 23:00:00,False,True
2019-06-01 23:15:00,4.547,2019-06-01 23:15:00,False,True


#### Traffic patterns are also different on holidays compared to normal working days therefore, the holiday data is also added

In [127]:
df_holidays = helper.get_national_holidays(year=2019)
df_holidays

,holiday,ds,lower_window,upper_window
0,New year,2019-01-01,0,1
1,Epiphany,2019-01-06,0,1
2,Easter Monday,2019-04-22,0,1
3,State Holiday,2019-05-01,0,1
4,Ascension Thursday,2019-05-30,0,1
5,Whit Monday,2019-06-10,0,1
6,Corpus Christi,2019-06-20,0,1
7,Assumption of Mary to Heaven,2019-08-15,0,1
8,National Holiday,2019-10-26,0,1
9,All Saints Day,2019-11-01,0,1


#### Depending upon what series is going on, the network traffic varies alot as well, so it is usefull to add this info. as well, there are many free sites from where such data can be donwloaded

In [128]:
match_dates = helper.get_matchdates()
conditions = df_data.ds.isin(match_dates.DateTimeAdj)

In [129]:
df_data['game_played'] = np.select(conditions, df_data.ds)

<ipython-input-129-d0a5b6b2e4a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['game_played'] = np.select(conditions, df_data.ds)


In [130]:
#df_data = df_data.drop(['ds'], axis=1)
df_data

,y,ds,week_days,week_ends,game_played
ds,,,,,
2019-01-02 00:00:00,15.076,2019-01-02 00:00:00,True,False,0
2019-01-02 00:15:00,17.461,2019-01-02 00:15:00,True,False,0
2019-01-02 00:30:00,9.163,2019-01-02 00:30:00,True,False,0
2019-01-02 00:45:00,12.535,2019-01-02 00:45:00,True,False,0
2019-01-02 01:00:00,13.531,2019-01-02 01:00:00,True,False,0
...,...,...,...,...,...
2019-06-01 22:45:00,7.253,2019-06-01 22:45:00,False,True,0
2019-06-01 23:00:00,7.670,2019-06-01 23:00:00,False,True,0
2019-06-01 23:15:00,4.547,2019-06-01 23:15:00,False,True,0


#### Lets seperate train from test. I added last 7 entries (quarters) as test data

In [131]:
test_window = 7 
max_rows = df_data.shape[0]

df_train = df_data[0:max_rows-test_window]
df_test = df_data[max_rows-test_window:max_rows]
df_train.shape, df_test.shape

((1090259, 5), (7, 5))

In [132]:
df_train

,y,ds,week_days,week_ends,game_played
ds,,,,,
2019-01-02 00:00:00,15.076,2019-01-02 00:00:00,True,False,0
2019-01-02 00:15:00,17.461,2019-01-02 00:15:00,True,False,0
2019-01-02 00:30:00,9.163,2019-01-02 00:30:00,True,False,0
2019-01-02 00:45:00,12.535,2019-01-02 00:45:00,True,False,0
2019-01-02 01:00:00,13.531,2019-01-02 01:00:00,True,False,0
...,...,...,...,...,...
2019-06-01 21:00:00,7.992,2019-06-01 21:00:00,False,True,0
2019-06-01 21:15:00,9.555,2019-06-01 21:15:00,False,True,0
2019-06-01 21:30:00,5.002,2019-06-01 21:30:00,False,True,0


In [133]:
m_prophet = Prophet(daily_seasonality=True, weekly_seasonality=False, holidays = df_holidays)
m_prophet.add_seasonality(name='weekly_days_season', period=7, fourier_order=3, condition_name='week_days')
m_prophet.add_seasonality(name='weekly_ends_season', period=7, fourier_order=3, condition_name='week_ends')
m_prophet.add_regressor('game_played')

In [ ]:
m_prophet.fit(df_train)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [ ]:
fcast_time=test_window
df_forecast = df_prophet.make_future_dataframe(periods= fcast_time, freq='D', include_history=True)

In [ ]:
fig = mdl_prophet.plot(df_forecast_w_history)
fig.set_size_inches(20, 16)
plt.xlabel('yhat', fontsize=24)
plt.ylabel('date', fontsize=24)
plt.rcParams['xtick.labelsize']=8
plt.rcParams['ytick.labelsize']=8
a = add_changepoints_to_plot(fig.gca(), mdl_prophet, df_forecast_w_history)